In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
usecase_name = "Track order's status"
project_name = "ShopeeWeb"
main_flow_prompt = """
Main flow:
Step 1: User clicks on the personal name and avatar section at the top right of the platform's page.
Step 2: System redirects user to their personal bought order page. There are a list of all orders user have choose to bought and their information and status. On the top of the list there is a bar with multiple tabs, each tab is named after a state of an order including All, Waiting for payment, Delivering, Waiting to deliver, Finished, Cancelled and Returned. There is a search bar below the status bar for searching order.
"""
prompt_all = """
Main flow:
Step 1: User clicks on the personal name and avatar section at the top right of the platform's page.
Step 2: System redirects user to their personal bought order page. There are a list of all orders user have choose to bought and their information and status. On the top of the list there is a bar with multiple tabs, each tab is named after a state of an order including All, Waiting for payment, Delivering, Waiting to deliver, Finished, Cancelled and Returned. There is a search bar below the status bar for searching order.

Alternative flow 1: User filter order by status
Step 3: User clicks on the tab with the status name they want system to filter the order.
Step 4: System shows only order with the corresponding chosen status.

Alternative flow 2: User search order
Step 3: User fills in one of these options of shop's name, order's ID or product's ID in the search bar.
Step 4: User press enter.
Step 5: System shows orders with keyword corresponding to user input.
"""
alt_prompt=["""Alternative flow 1: User filter order by status
Step 3: User clicks on the tab with the status name they want system to filter the order.
Step 4: System shows only order with the corresponding chosen status.""",
"""Alternative flow 2: User search order
Step 3: User fills in one of these options of shop's name, order's ID or product's ID in the search bar.
Step 4: User press enter.
Step 5: System shows orders with keyword corresponding to user input."""]
exc_prompt = []

In [3]:
#Prompt
MAIN_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow of a use case.
Then you predict all scenarios that can happen in this flow.

Rules to predict scenarios:
- Stay close to the details described in the flow.
- Choose important cases to generate, important scenario is the scenario that users are more likely to encounter it.
- Limit the appearance of scenarios that are hard to happen. 
- A scenario encompasses a whole function, not just verifying individual steps.
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
I only need scenarios's name for the output, I do not need the steps to go with it.
Example:
The scenario's name structure should start with who or what role can do this action + what they are testing in this scenario (including information about they can or cannot do the action. If they cannot, why).
Users successfully create a new account.
Users cannot create a new account if the username field is blank.
Users can navigate to Add New Lesson Page via Navbar Option.
"""

SUB_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow and one alternative or exception flow of a use case.
Then you predict all scenarios that can lead user from the main flow to change to the alternative or exception flow mentioned for creating test cases.

Rules to predict scenarios:
- A scenario encompasses a whole function, not just verifying individual steps.
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
- Do not generate scenarios with user analysis. (Example: User accidentally do A and user intentionally do A is the same scenario, so do not consider about "accidentally" or "intentionally" in scenario)
- Do not choose another option that is not chosen by the flow, eventhough it is mentioned (Example: A pop up with OK and Cancel, the flow only has step choose OK. Do not generate scenario that press Cancel)
- Do not generate scenario to test only the main flow.
I only need scenarios's name for the output, I do not need the steps to go with it.
Example:
The scenario's name structure should start with who or what role can do this action + what they are testing in this scenario (including information about they can or cannot do the action. If they cannot, why).
Users successfully create a new account.
Users cannot create a new account if the username field is blank.
Users can navigate to Add New Lesson Page via Navbar Option.
"""

EXTRACT_CONDITION_SYSTEM_PROMPT="""
Given use case flows of a feature.
Your task is to identify all the interactive elements within the feature. 
For each interactive element:
Identify what type of that element (button,buttons, icon,scroll, text field,text area, tab, radio buttons, menu, combobox, sliders, switches, dialog, link, form,rating, filter).
Identify all the conditions mentioned in the use case of that element that would make the element valid and the conditions that would make the element invalid based on the description of the use case flow.
Do not arbitrarily create additional conditions that not mention in the use case flow.
Return the element extracted in json format.
The JSON format should follow the following structure:
{"Name of interactive element": {"condition": {valid:"conditions that make element valid", invalid: "conditions that make element invalid"}, "type": "element type"}}
Examples of output json format template: 
{"Username": {"condition": {"valid": "must be over 8 characters and below 30 characters, must be entered", racter, empty"}, "type": "text field"}}
{"Search button": {"condition": {"valid": "", "invalid": ""}, "type": "text field"}}
"""

GEN_SCENARIO_FOR_CONDITION_PROMPT="""
Given a list of interaction element for input value, their extracted conditions and use c"invalid": "below 8 characters, over 30 chaase specification.
For each given invalid condition:
- Generate a test scenario that test that condition.
Do not generate test scenario to test element/condition that not mention in the given element list.
Do not generate test scenario to test valid conditions.
I only need scenarios's name for the output, I do not need the steps to go with it.
Example:
The scenario's name structure should start with who or what role can do this action + what they are testing in this scenario (including information about they can or cannot do the action. If they cannot, why).
Users successfully create a new account.
Users cannot create a new account if the username field is blank.
Users can navigate to Add New Lesson Page via Navbar Option.
"""


FILTER_SYSTEM_PROMPT="""
Your task is to read all of this scenarios generation from many sources.
Then you remove all the same copies because there are many duplicate scenarios in meanings inside the input.
If a scenario from the flow scenario is duplicate with a scenario from the condition scenario, delete the one from the flow scenario and keep the one from the condition.
Make sure every scenarios in the response is unique.
Return scenario name only.
"""

FILTER_SYSTEM_PROMPT2="""
You will be given a list of elements, the conditions linked to each elements and the corresponding scenario to test for the elements.
Your task is to identify all the duplicate scenarios and delete it.
Make sure every scenarios in the response is unique.
Return scenario name only.
There are scenarios with different wording but test the same condition.
Example:
1. User cannot create account because of invalid username.
2. User cannot create account because username has more than 8 characters.
3. User cannot create account because username has less than 5 characters.
Scenario 1 is consider to be duplicate with scenario 2 and scenario 3 because it does not make it clear what is invalid, so there is chance for scenario 1 to produce test cases duplicate with 2 and 3. But if 2 and 3 is not exists, scenario 1 is consider to be unique.
"""

Prompt_Extract_Action = """
Read this list of test scenarios.
For each test scenario:
    Define the action of user.     For example:  "successfully create", "cannot create", "cancels", "navigates". Keep the number of action minimum. 
    For each action: Group the test scenario with that action.
Return the result in json format.
The JSON format should follow the following structure:
{
  "action": [test scenarios],
  }

An example of a test scenario: 
{
  "successfully create": ["user successfully create object"],
  "cannot create": ["user cannot create object with invalid data","user cannot create object with missing data"]
  }
"""
FILTER_SYSTEM_PROMPT3= """You will be given a list of scenarios group by action.
Your task is to identify all the duplicate scenarios and delete it.
Make sure every scenarios in the response is unique. 
There are also cases that different actions could be the same meaning with the context. Please check through all the actions and their corresponding scenarios to see if two different words are being used to refer a same action.
Return scenario name only.
There are scenarios with different wording but test the same condition.

Example for conditional duplicate:
Create:
1. User cannot create account because of invalid username.
2. User cannot create account because username has more than 8 characters.
3. User cannot create account because username has less than 5 characters.
Scenario 1 is consider to be duplicate with scenario 2 and scenario 3 because it does not make it clear what is invalid, so there is chance for scenario 1 to produce test cases duplicate with 2 and 3. But if 2 and 3 is not exists, scenario 1 is consider to be unique.

Example for same goal:
Exit:
1. User exits the page.
2. User press exits and confirm the exit.
3. User press exits but cancel the exit.
Leave:
4. User leaves the page.
Scenario 1 is consider to be duplicate with scenario 2 and 4, because the goal is the same and the test steps is predicted to be the same for the test cases created after this phase. For scenario 4, even though the verb is different but they are mentioning the same action. But scenario 1 and scenario 3 is different from each other because their final results and steps are expected to be different.

"""

In [4]:
def pop_list_values(list1, list2):
    # Create a copy of list1 to avoid modifying the original list during iteration
    list1_copy = list1[:]
    
    # Iterate over each element in list2 and remove it from list1_copy if it exists
    for value in list2:
        while value in list1_copy:
            list1_copy.remove(value)
    
    return list1_copy

def get_key_value_by_index(json_obj, index):
    # Convert JSON object to dictionary if it's a JSON string
    if isinstance(json_obj, str):
        json_obj = json.loads(json_obj)
    
    # Get all key-value pairs as a list of tuples
    key_value_pairs = list(json_obj.items())
    
    # Check if the index is within the range
    if index < 0 or index >= len(key_value_pairs):
        raise IndexError("Index out of range")
    
    # Return the key-value pair at the specified index
    return key_value_pairs[index]

def list_to_string(input_list):
    return '\n'.join(map(str, input_list))
def string_to_list(string_obj):
    return string_obj.splitlines()

def write_string_to_file(filename, content):
    try:
        with open(filename, 'w') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [5]:
for i in range(1,4):
  print("TIME "+str(i))
  promptMainScenario = [
      { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt}
    ]
  main_gpt_response = ask(promptMainScenario, client, model)
  filtercontent = main_gpt_response
  for alt in alt_prompt:
    promptSubScenario = [
        { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
        { "role": "user", "content": main_flow_prompt +"\n\n"+alt}
      ]
    sub_gpt_response = ask(promptSubScenario, client, model)
    filtercontent+=  "\n\n" + sub_gpt_response
  for exc in exc_prompt:
    promptSubScenario = [
        { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
        { "role": "user", "content": main_flow_prompt +"\n\n"+exc}
      ]
    sub_gpt_response = ask(promptSubScenario, client, model)
    filtercontent+=  "\n\n" + sub_gpt_response

  promptExtractCondition = [
      { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
      { "role": "user", "content": prompt_all}
    ]
  gpt_response = askJSON(promptExtractCondition, client, model)
  full_elements = json.loads(gpt_response)
  condition_element = {key: value for key, value in full_elements.items() 
                                if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
  if(len(condition_element)!= 0):
    promptScenarioForCondition = [
        { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
        { "role": "user", "content": prompt_all + '\n Element:' + str(condition_element)}
      ]
    condition_scenario_response = ask(promptScenarioForCondition, client, model)
    print(condition_scenario_response)
    filtercontent += "\n" + condition_scenario_response

  print(filtercontent)
  element_list = []
  for element in condition_element:
      print(element)
      element_temp = element.replace(" field","")
      element_list.append(element_temp.lower())


  test_scenario_filtercontent = string_to_list(filtercontent.lower())

  print(test_scenario_filtercontent)
  test_scenario_list = []
  print(element_list)
  for element in element_list:
      temp = []
      for tc in test_scenario_filtercontent:
          if element in tc:
              temp.append(tc)
      test_scenario_list.append(temp)

  for element in test_scenario_list:
      test_scenario_filtercontent = pop_list_values(test_scenario_filtercontent,element)

  for index in range(len(test_scenario_list)):
      test_scenario_cluster = test_scenario_list[index]
      if(len(test_scenario_cluster) != 1):
          print(len(test_scenario_cluster))
          print((test_scenario_cluster))

          promptFilter = [
          { "role": "system", "content": FILTER_SYSTEM_PROMPT2},
          { "role": "user", "content":"Condition" +str(get_key_value_by_index(condition_element, index))+"\n"+ str(test_scenario_cluster)}
          ]
          filter_gpt_response = ask(promptFilter, client, model)
          print("\n" + "Final result:" + filter_gpt_response)

          test_scenario_filtercontent.append(filter_gpt_response)

  promptFilter = [
      { "role": "system", "content": FILTER_SYSTEM_PROMPT},
      { "role": "user", "content": list_to_string(test_scenario_filtercontent)}
    ]
  filter_gpt_response = ask(promptFilter, client, model)
  print("\n" + "Phase 2 result:" + filter_gpt_response)

  promptExtractCondition = [
      { "role": "system", "content": Prompt_Extract_Action},
      { "role": "user", "content": filter_gpt_response}
    ]
  gpt_response = askJSON(promptExtractCondition, client, model)
  print(gpt_response)
  group_testscearnio = json.loads(gpt_response)

  promptExtractCondition = [
      { "role": "system", "content": FILTER_SYSTEM_PROMPT3},
      { "role": "user", "content": str(group_testscearnio)}
    ]
  gpt_response = ask(promptExtractCondition, client, model)
  print("\n" + "Phase 3 result:"+ gpt_response)
  write_string_to_file(f"D:\\Dissertation-GPT\\evaluate\\ResultSet\\5.20\\filter-old-condition-new\\{project_name}\\{usecase_name}-{i}.txt", gpt_response)

TIME 1
Users cannot filter orders if the search bar input is empty.
User successfully navigates to personal bought order page and views all orders.
User filters orders by selecting the "Waiting for payment" tab.
User filters orders by selecting the "Delivering" tab.
User filters orders by selecting the "Waiting to deliver" tab.
User filters orders by selecting the "Finished" tab.
User filters orders by selecting the "Cancelled" tab.
User filters orders by selecting the "Returned" tab.
User successfully searches for an order using the search bar.

Users successfully navigate to their personal bought order page.
Users successfully filter orders by status.

Users can access their personal bought order page.
Users can search for orders using shop's name.
Users can search for orders using order's ID.
Users can search for orders using product's ID.
Users cannot filter orders if the search bar input is empty.
Search bar
['user successfully navigates to personal bought order page and views all